In [1]:
%cd ..
import gym
import gym_game
import matplotlib.pyplot as plt
env = gym.make('Parkme')
reset, _ = env.reset()
n_actions = env.action_space.n
states_dim = env.observation_space.shape[0]
print(n_actions, states_dim)

/home/yvovaa/fork/HSE_ML_P2024
9 7


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [2]:
import torch.nn as nn
import torch
from torch.optim import Adam
class MyGRU(nn.Module):
    def __init__(self, states_dim, actions_dim, hidden_dim) -> None:
        super().__init__()
        self.states_dim = states_dim
        self.hidden_dim = hidden_dim
        self.gru = nn.GRU(
            states_dim,
            hidden_dim,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_dim, actions_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        output = self.fc(output)
        output = self.softmax(output)
        return output, hidden
    
        


In [3]:
hidden_dim = 128
model = MyGRU(states_dim, n_actions, hidden_dim)
optimizer = Adam(model.parameters())

In [4]:
import numpy as np
import random
def generate_session(env, agent, exploration_rate=0.8):
    states, actions = [], []
    total_reward = 0
    s, _ = env.reset()
    hidden = torch.zeros(1, 128)
    for step in range(3000):
        if random.random() > exploration_rate:
            action = np.random.choice(list(range(n_actions)))
        else:
            with torch.no_grad():
                probs, hidden = agent(torch.FloatTensor(s).unsqueeze(0), hidden)
                action = torch.argmax(probs.squeeze(0)).item()   
        new_s, r, terminated, truncated, _ = env.step(action)
        states.append(s)
        actions.append(action)
        total_reward += r

        s = new_s
        if terminated or truncated:
            return states, actions, total_reward / step 
    return states, actions, total_reward / 3000

In [5]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile):
    reward_threshold = np.percentile(rewards_batch, percentile)
    condition = np.array(rewards_batch) >= reward_threshold
    return np.array(states_batch)[condition], np.array(actions_batch)[condition]

In [6]:
from IPython.display import clear_output

def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])
    clear_output(True)
    print("mean reward = %.3f, threshold=%.3f" % (mean_reward, threshold))
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label="Mean rewards")
    plt.plot(list(zip(*log))[1], label="Reward thresholds")
    plt.legend()
    plt.grid()
    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines(
        [np.percentile(rewards_batch, percentile)],
        [0],
        [100],
        label="percentile",
        color="red",
    )
    plt.legend()
    plt.grid()
    plt.show()


In [7]:
percentile = 85
log = []
n_epochs = 80
best_reward = -1e12
worst_reward = 1e12

In [8]:
%%time
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
criterion = CrossEntropyLoss()
for i in tqdm(range(n_epochs)):
    sessions = [generate_session(env, model) for _ in range(5)]
    states_batch, actions_batch, rewards_batch = zip(*sessions)
    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)
    hidden = torch.zeros(1, len(elite_states), model.hidden_dim)
    outps, _ = model(torch.FloatTensor(elite_states), hidden)
    optimizer.zero_grad()
    loss = criterion(outps.float(), torch.FloatTensor(elite_actions))
    loss.backward()
    optimizer.step()
    mean = np.mean(rewards_batch)
    if mean > best_reward:
        best_reward = mean
        torch.save(model.state_dict(), "gru.pt")
    if mean < worst_reward:
        worst_reward = mean
    show_progress(
        rewards_batch, log, percentile, reward_range=[worst_reward, best_reward]
    )

  0%|          | 0/80 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


: 

Я НЕ ПОНИМАЮ, в чем проблема. При генерации последней сессии ядро успешно падает